In [1]:
import sys
sys.path.append("c:/users/david/desktop/sandbox")
import pandas as pd
import numpy as np
from pyrecs.train import lightfm_wrapper

%load_ext autoreload
%autoreload 2

C:\Users\David\Desktop\Sandbox\pyrecs\env\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


# Pure CF

In [2]:
df = pd.read_csv('../data/h&m/transactions_train_with_dates.csv')
df.drop_duplicates(['customer_id', 'article_id'], keep='first', inplace=True)

for col in ['customer_id','article_id']:
    df[col] = df[col].astype(str)
    
print(df.shape)
df.head()

(27306439, 9)


,t_dat,customer_id,article_id,price,sales_channel_id,t_dat_year,t_dat_week,customer_ind,article_ind
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,2018,38,0,0
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,2018,38,0,1
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,2018,38,1,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,2018,38,1,3
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,2018,38,1,4


In [3]:
train_df = df[~((df['t_dat_year']==2020) & (df['t_dat_week'].isin([38,39])))].copy()
test_df = df[(df['t_dat_year']==2020) & (df['t_dat_week'].isin([38,39]))].copy()

train_df.shape, test_df.shape

((27053842, 9), (252597, 9))

In [4]:
train_df['customer_id'].nunique(), test_df['customer_id'].nunique(), len(set(train_df['customer_id'].unique()) & set(test_df['customer_id'].unique()))

(1355461, 80430, 73610)

In [5]:
train_df['article_id'].nunique(), test_df['article_id'].nunique(), len(set(train_df['article_id'].unique()) & set(test_df['article_id'].unique()))

(103629, 19177, 18259)

In [6]:
df.dtypes

t_dat                object
customer_id          object
article_id           object
price               float64
sales_channel_id      int64
t_dat_year            int64
t_dat_week            int64
customer_ind          int64
article_ind           int64
dtype: object

In [7]:
model_kwargs = {
    'no_components':30,
    'learning_rate':0.05,
    'loss':'warp',
    'random_state':42
}
train_kwargs = {
    'num_epochs':2,
    'num_threads':1,
    'eval_epochs':'all',
    'plot':True
}
lfm = lightfm_wrapper.LightFM(users_col='customer_id', items_col='article_id', interactions_type='ones',
                              model_kwargs=model_kwargs, train_kwargs=train_kwargs,
                              n_recs=12, tfrs_prediction_batch_size=512)

In [8]:
lfm.run(train_df=train_df, test_df=test_df)

Training:   0%|                                                                                  | 0/2 [32:56<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
lfm.train_evaluations, lfm.test_evaluations